In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from collections import defaultdict

from surprise.model_selection import KFold

from surprise.model_selection.validation  import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans

import os
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

from surprise.model_selection import train_test_split

from sklearn.metrics import  roc_curve
from sklearn.metrics import roc_auc_score
from collections import defaultdict

from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.matrix_factorization import SVD

import seaborn as sns

import csv
import re


In [65]:
bike_data = pd.read_csv('../input/project4set2/day.csv')

bike_data

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,364,1432,1796


In [66]:
video_data = pd.read_csv('../input/project4set2/transcoding_mesurment.tsv', sep='\t')
video_data

,id,duration,codec,width,height,bitrate,framerate,i,p,b,...,p_size,b_size,size,o_codec,o_bitrate,o_framerate,o_width,o_height,umem,utime
0,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,176,144,22508,0.612
1,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,320,240,25164,0.980
2,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,480,360,29228,1.216
3,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,640,480,34316,1.692
4,04t6-jw9czg,130.35667,mpeg4,176,144,54590,12.000000,27,1537,0,...,825054,0,889537,mpeg4,56000,12.00,1280,720,58528,3.456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,ZWEN-71BqPs,972.27100,h264,480,360,278822,29.000000,560,28580,0,...,26561730,0,33886358,flv,242000,24.00,640,480,88692,1.552
68780,zWQN-bqqg0o,129.88100,vp8,640,480,639331,30.162790,36,3855,0,...,9503846,0,10379630,mpeg4,539000,29.97,1920,1080,107524,18.557
68781,zX17-vi0sqQ,249.68000,vp8,320,240,359345,25.068274,129,6113,0,...,9456514,0,11215178,flv,539000,12.00,176,144,88708,0.752
68782,zyiT-TzxIpk,183.62334,h264,1280,720,2847539,29.000000,98,5405,0,...,60113035,0,65359329,mpeg4,539000,12.00,320,240,88724,5.444


In [67]:

bike_data_df = pd.DataFrame(data=bike_data)
video_data_df = pd.DataFrame(data=video_data)


In [68]:
# one hot encoding - bike_data
from sklearn.preprocessing import OneHotEncoder 


#http://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example

#one hot encode weathersit for sure, season probably, weekday (0-6) maybe
# label encode the date into days of the month
encoded_bike_data = bike_data_df.copy()

encoded_bike_data['dteday'] = pd.to_datetime(bike_data_df['dteday']).dt.strftime('%d')
encoded_bike_data['dteday'] = pd.to_numeric(encoded_bike_data['dteday'])

# use pd.concat to join the new columns with your original dataframe
df_season = pd.concat([encoded_bike_data,pd.get_dummies(encoded_bike_data['season'], prefix='season')],axis=1)

# now drop the original 'country' column (you don't need it anymore)
df_season.drop(['season'],axis=1, inplace=True)

df_weathersit = pd.concat([df_season, pd.get_dummies(df_season['weathersit'], prefix = 'weathersit')], axis=1)
df_weathersit.drop(['weathersit'], axis=1, inplace=True)


df_holiday = pd.concat([df_weathersit, pd.get_dummies(df_weathersit['holiday'], prefix = 'holiday')], axis=1)
df_holiday.drop(['holiday'], axis=1, inplace=True)

df_workday = pd.concat([df_holiday, pd.get_dummies(df_holiday['workingday'], prefix = 'workingday')], axis=1)
df_workday.drop(['workingday'], axis=1, inplace=True)
bike_data_proc_df = df_workday
print(bike_data_proc_df.columns)
bike_no_f_scaling = bike_data_proc_df.copy()
bike_data_proc_df


Index(['instant', 'dteday', 'yr', 'mnth', 'weekday', 'temp', 'atemp', 'hum',
       'windspeed', 'casual', 'registered', 'cnt', 'season_1', 'season_2',
       'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3',
       'holiday_0', 'holiday_1', 'workingday_0', 'workingday_1'],
      dtype='object')


,instant,dteday,yr,mnth,weekday,temp,atemp,hum,windspeed,casual,...,season_2,season_3,season_4,weathersit_1,weathersit_2,weathersit_3,holiday_0,holiday_1,workingday_0,workingday_1
0,1,1,0,1,6,0.344167,0.363625,0.805833,0.160446,331,...,0,0,0,0,1,0,1,0,1,0
1,2,2,0,1,0,0.363478,0.353739,0.696087,0.248539,131,...,0,0,0,0,1,0,1,0,1,0
2,3,3,0,1,1,0.196364,0.189405,0.437273,0.248309,120,...,0,0,0,1,0,0,1,0,0,1
3,4,4,0,1,2,0.200000,0.212122,0.590435,0.160296,108,...,0,0,0,1,0,0,1,0,0,1
4,5,5,0,1,3,0.226957,0.229270,0.436957,0.186900,82,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,27,1,12,4,0.254167,0.226642,0.652917,0.350133,247,...,0,0,0,0,1,0,1,0,0,1
727,728,28,1,12,5,0.253333,0.255046,0.590000,0.155471,644,...,0,0,0,0,1,0,1,0,0,1
728,729,29,1,12,6,0.253333,0.242400,0.752917,0.124383,159,...,0,0,0,0,1,0,1,0,1,0
729,730,30,1,12,0,0.255833,0.231700,0.483333,0.350754,364,...,0,0,0,1,0,0,1,0,1,0


In [69]:
# one hot encoding - video data
video_data_noid = video_data_df.copy()
video_data_noid.drop(['id'], axis=1, inplace=True)


df_codec = pd.concat([video_data_noid, pd.get_dummies(video_data_noid['codec'], prefix = 'codec')], axis=1)
df_codec.drop(['codec'], axis=1, inplace=True)


df_o_codec = pd.concat([df_codec, pd.get_dummies(df_codec['o_codec'], prefix = 'o_codec')], axis=1)
df_o_codec.drop(['o_codec'], axis=1, inplace=True)

video_data_proc_df = df_o_codec.copy()
video_no_f_scaling = video_data_proc_df.copy()
video_data_proc_df

,duration,width,height,bitrate,framerate,i,p,b,frames,i_size,...,umem,utime,codec_flv,codec_h264,codec_mpeg4,codec_vp8,o_codec_flv,o_codec_h264,o_codec_mpeg4,o_codec_vp8
0,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,22508,0.612,0,0,1,0,0,0,1,0
1,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,25164,0.980,0,0,1,0,0,0,1,0
2,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,29228,1.216,0,0,1,0,0,0,1,0
3,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,34316,1.692,0,0,1,0,0,0,1,0
4,130.35667,176,144,54590,12.000000,27,1537,0,1564,64483,...,58528,3.456,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68779,972.27100,480,360,278822,29.000000,560,28580,0,29140,7324628,...,88692,1.552,0,1,0,0,1,0,0,0
68780,129.88100,640,480,639331,30.162790,36,3855,0,3891,875784,...,107524,18.557,0,0,0,1,0,0,1,0
68781,249.68000,320,240,359345,25.068274,129,6113,0,6242,1758664,...,88708,0.752,0,0,0,1,1,0,0,0
68782,183.62334,1280,720,2847539,29.000000,98,5405,0,5503,5246294,...,88724,5.444,0,1,0,0,0,0,1,0


In [70]:
#question 7 - bike data
from sklearn.preprocessing import StandardScaler

#standardize all feature columns

col_names = bike_data_proc_df.columns
features = bike_data_proc_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

#assign scaled results to the dataset copy
bike_data_proc_df[col_names] = features
print(bike_data_proc_df)

print(np.mean(bike_data_proc_df['atemp']))


      instant    dteday        yr      mnth   weekday      temp     atemp  \
0   -1.729683 -1.674108 -1.001369 -1.600161  1.498809 -0.826662 -0.679946   
1   -1.724944 -1.560522 -1.001369 -1.600161 -1.496077 -0.721095 -0.740652   
2   -1.720205 -1.446936 -1.001369 -1.600161 -0.996930 -1.634657 -1.749767   
3   -1.715466 -1.333351 -1.001369 -1.600161 -0.497782 -1.614780 -1.610270   
4   -1.710728 -1.219765 -1.001369 -1.600161  0.001366 -1.467414 -1.504971   
..        ...       ...       ...       ...       ...       ...       ...   
726  1.710728  1.279122  0.998633  1.588660  0.500513 -1.318665 -1.521108   
727  1.715466  1.392707  0.998633  1.588660  0.999661 -1.323224 -1.346690   
728  1.720205  1.506293  0.998633  1.588660  1.498809 -1.323224 -1.424344   
729  1.724944  1.619879  0.998633  1.588660 -1.496077 -1.309558 -1.490049   
730  1.729683  1.733465  0.998633  1.588660 -0.996930 -1.528225 -1.540482   

          hum  windspeed    casual  ...  season_2  season_3  season_4  \
0 

In [71]:
#question 7 continued

from sklearn.preprocessing import StandardScaler


video_col_names = video_data_proc_df.columns
video_features = video_data_proc_df[video_col_names]
video_scaler = StandardScaler().fit(video_features.values)
video_features = video_scaler.transform(video_features.values)

#assign scaled results to the dataset copy
video_data_proc_df[video_col_names] = video_features
print(video_data_proc_df)

#check that mean of a column is approximately 0
print(np.mean(video_data_proc_df['p']))


       duration     width    height   bitrate  framerate         i         p  \
0     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
1     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
2     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
3     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
4     -0.543270 -0.969273 -1.116197 -0.583333  -1.555936 -0.871457 -0.822060   
...         ...       ...       ...       ...        ...       ...       ...   
68779  2.387620 -0.312921 -0.218492 -0.378671   0.797072  5.416577  3.628857   
68780 -0.544926  0.032528  0.280232 -0.049625   0.958016 -0.765280 -0.440548   
68781 -0.127879 -0.658369 -0.717217 -0.305176   0.252873  0.331882 -0.068911   
68782 -0.357837  1.414323  1.277682  1.965863   0.797072 -0.033839 -0.185438   
68783  0.028544 -0.969273 -1.116197 -0.582738  -1.555936 -0.470344 -0.503255   

              b    frames    i_size  ..

In [73]:
#question 8 feature selection

from sklearn.feature_selection import f_regression, mutual_info_regression


#separate the matrices into features and targets
bike_target = bike_data_proc_df.cnt
video_target = video_data_proc_df.utime

bike_feature_mtx = bike_data_proc_df.drop(['registered','casual','cnt'], axis=1, inplace=False)

video_feature_mtx = video_data_proc_df.drop(['utime'], axis=1, inplace=False)



In [88]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from scipy import stats

forest = RandomForestRegressor()


parameter_space = {
    'n_estimators': [10, 20, 50, 100, 200],
    'max_depth': [2, 4, 8, 16, 32, 64],
    'max_features': [1, 2, 5, 10, 15],
    'oob_score': [True]
}
forestclf_bike = GridSearchCV(forest, parameter_space, n_jobs=-1, cv=10, return_train_score=True,scoring='neg_mean_squared_error')
forestclf_bike.fit(bike_feature_mtx, bike_target)

print('Bike Data: Best Random Forest parameters found:\n', forestclf_bike.best_params_)
print(forestclf_bike.best_score_)
bike_train_MSE = -1* forestclf_bike['test_score']
bike_test_MSE = -1* forestclf_bike['train_score']

bike_train_RMSE = np.sqrt(bike_train_MSE)
bike_test_RMSE = np.sqrt(bike_test_MSE)

print("Bike Training Data Random Forest with optimal hyper-parameters, RMSE: " +str(np.mean(bike_train_RMSE)))
print("Bike Test Data Random Forest with optimal hyper-parameters, RMSE: " +str(np.mean(bike_test_RMSE)))

Bike Data: Best Random Forest parameters found:
 {'max_depth': 16, 'max_features': 5, 'n_estimators': 20, 'oob_score': True}
-0.19296533529279025


TypeError: 'GridSearchCV' object is not subscriptable

In [90]:

from sklearn.tree import export_graphviz
best_bike_forest = RandomForestRegressor(n_estimators=5, max_features=5, max_depth=16, bootstrap=True, oob_score=True)
best_bike_forest.fit(bike_feature_mtx, bike_target)
tree = best_bike_forest.estimators_[0]
predicted = tree.predict(bike_feature_mtx)
export_graphviz(tree, out_file='rf.dot', max_depth=4)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:815: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [95]:
print("Best Bike Test RMSE Random forest: "+str(np.sqrt(-1* forestclf_bike.best_score_)))
print("Best Bike Test OOB score: "+str(best_bike_forest.oob_score_))



Best Bike Test RMSE Random forest: 0.4392781980622192
Best Bike Test OOB score: 0.7267552660736631


In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()


parameter_space = {
    'n_estimators': [10, 20, 50, 100, 200],
    'max_depth': [2, 4, 8, 16, 32, 64],
    'max_features': [1, 2, 5, 10, 15],
}
forestclf_video = GridSearchCV(forest, parameter_space, n_jobs=-1, cv=10, return_train_score=True,scoring='neg_mean_squared_error')
forestclf_video.fit(video_feature_mtx, video_target)

print('Video Data: Best Random Forest parameters found:\n', forestclf_video.best_params_)
print(forestclf_video.best_score_)
video_train_MSE = -1* forestclf_video['test_score']
video_test_MSE = -1* forestclf_video['train_score']

video_train_RMSE = np.sqrt(video_train_MSE)
video_test_RMSE = np.sqrt(video_test_MSE)

print("Video Training Data Random Forest with optimal hyper-parameters, RMSE: " +str(np.mean(video_train_RMSE)))
print("Video Test Data Random Forest with optimal hyper-parameters, RMSE: " +str(np.mean(video_test_RMSE)))

In [ ]:
print("Best Video Test RMSE Random forest: "+str(np.sqrt(-1* forestclf_video.best_score_)))

In [ ]:
#comment##